# Case studies

This notebook works with:

- Trigger data for clusters
- Coordinates data for case studies
  * Reads processed data for clusters
  * Gets nearest cluster that contains the coordinates of each of the case study locations
  * Assigns triggers for each site

In [82]:
import sys #
import os
from scipy.stats import norm
from scipy.signal import butter, filtfilt

gp = '/home/jupyter-gabriel/'#'I:/My Drive/'
sys.path.append(gp + "projects/suyana")
path = 'suyana/cluster/'
from scripts.utils.functions import *
#from scripts.utils.data_cleaning import *

In [83]:
# Data primary path
dpp = 'suyana/'
# Data inputs path
dip = 'suyana/inputs/'
# Data features path
dfp = 'suyana/features/'
# Data outputs path
dop = 'suyana/outputs/'
# Data reports path
drp = 'suyana/reports/'
# Data models path
dmp = 'suyana/models/'
# Data displays path
ddp = 'suyana/displays/'
# Data geometries path
dgp = 'suyana/geometries/'

In [85]:
df_case = pd.read_csv(dip + 'drought_case_study_locations_2024.csv')

In [86]:
gdf = gpd.read_file(dgp + 'dict_canton.gpkg')

In [51]:
gdf['new_id'] = gdf.apply(lambda row: str(row['new_id_canton']) +str('-') + str(row['new_id_cluster']), axis = 1)

In [ ]:
gdf.to_file(dgp + 'dict_canton.gpkg')

# Explore the points on the current polygon structure

In [ ]:
m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=7.4)

# Add canton boundaries with thicker blue lines
for _, row in gdf.iterrows():
    folium.GeoJson(
        row["geometry"].__geo_interface__,
        tooltip=f"Canton: {row['new_id_canton']}",
        style_function=lambda x: {
            "color": "black",
            "weight": 1,  # Thicker boundaries
            "fillColor": "blue",
            "fillOpacity": 0.5,
        },
    ).add_to(m)

for idx, row in df_case.iterrows():
    folium.Marker(
        location=[row['lat_corrected'], row['lon_corrected']],
        popup=f"Empresa: {row['Empresa']}<br>Municipio/Zona: {row['Municipio/ Zona']}<br>Propiedad: {row['Propiedad']}<br>Lat: {row['lat_corrected']:.2f}<br>Lon: {row['lon_corrected']:.2f}",
        tooltip=row['Empresa'],
        icon=folium.Icon(color="red")
    ).add_to(m)

m.save(ddp + 'drought_case_study_locations_2024.html')
m

/tmp/ipykernel_151492/2200417750.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=7.4)


# Identify each case study site's cluster

In [53]:
# A first alternative to do this is to use geodataframe joins
# First create a gdf with points using latitude and longitude pairs
df_geo = gpd.GeoDataFrame(df_case, geometry=gpd.points_from_xy(df_case.lon_corrected, df_case.lat_corrected), crs=gdf.crs)

# Then make a spatial join
df_geo = gpd.sjoin(df_geo, gdf, how="left", predicate="within")

df_geo = df_geo[['lon_corrected', 'lat_corrected', 'id_canton',"id_cluster", 'new_id']].drop_duplicates()\

df_case = df_case.merge(
    df_geo,
    how = 'left',
    on = ['lon_corrected', 'lat_corrected']
)
df_case


,Empresa,lat,lon,lat_corrected,lon_corrected,extension,Municipio/ Zona,Propiedad,Disponible en Planet?,ID pixel ERA5,planet_nearest_lat,planet_nearest_lon,planet_distance_km,era5_nearest_lat,era5_nearest_lon,era5_distance_km,id_canton,id_cluster,new_id
0,CARGILL,-63.650000,-16.566667,-16.566667,-63.650000,NaN,Colonia Piraí,Propiedad Agropecuaria La Bendecida,NaN,NaN,-17.670963,-62.100445,205.408268,-16.0,-63.0,93.720475,210.0,1.0,104-5
1,CARGILL,-17.983333,-60.983333,-17.983333,-60.983333,NaN,San José de Chiquitos,NaN,NaN,NaN,-17.492983,-61.923526,113.524822,-18.0,-61.0,2.557912,NaN,NaN,NaN
2,ALICORP,-61.742474,-17.327587,-17.327587,-61.742474,"2,928.55",CENTRO,Predio A,NaN,NaN,-17.492983,-61.923526,26.594188,-17.3,-61.7,5.453395,45.0,4.0,125-4
3,ALICORP,-62.195930,-17.129001,-17.129001,-62.195930,"1,988.83",CENTRO,Predio B,NaN,NaN,-17.492983,-61.923526,49.742420,-17.0,-62.0,25.289276,49.0,2.0,124-0
4,ALICORP,-62.449288,-16.756541,-16.756541,-62.449288,508.16,ESTE NORTE,Predio C,NaN,NaN,-17.492983,-61.923526,99.132132,-16.7,-62.3,17.095634,51.0,4.0,111-5
5,ALICORP,-63.003981,-16.592095,-16.592095,-63.003981,149.29,ESTE NORTE,Predio D,NaN,NaN,-17.537478,-61.967756,152.260539,-16.3,-62.7,45.889481,50.0,2.0,110-2
6,ALICORP,-61.816796,-17.582146,-17.582146,-61.816796,"6,469.99",ESTE SUR,Predio E,NaN,NaN,-17.492983,-61.923526,15.044988,-17.4,-61.6,30.640579,167.0,4.0,147-5
7,ALICORP,-61.746853,-17.596799,-17.596799,-61.746853,"2,521.63",ESTE SUR,Predio F,NaN,NaN,-17.492983,-61.923526,22.002671,-17.4,-61.6,26.859050,164.0,3.0,139-3
8,ALICORP,-61.177602,-16.130211,-16.130211,-61.177602,"4,225.64",ESTE EXPANSION,Predio G,NaN,NaN,-17.492983,-61.923526,171.073167,-16.9,-62.1,130.369622,NaN,NaN,NaN
9,ALICORP,-61.025796,-15.253538,-15.253538,-61.025796,"1,862.52",ESTE EXPANSION,Predio H,NaN,NaN,-17.492983,-61.923526,266.795375,-16.6,-62.4,209.775371,NaN,NaN,NaN


In [ ]:
df_case.to_csv(dip + 'drought_case_study_clusters_2024.csv', index=False)